In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


importing libraries

In [5]:
#!pip3 install transformers
#!pip3 install sentencepiece

In [6]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [7]:

import pandas as pd
#from datasets import load_dataset
from datasets import load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


from tqdm import tqdm
import torch


In [8]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]


In [47]:
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, 
                               batch_size=8, device=device, 
                               column_text="article", 
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        
        inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                        padding="max_length", return_tensors="pt")
        
        # Filter out examples with a shorter sequence length
        #mask = inputs['attention_mask'].sum(dim=1) > 0
        #inputs = {k: v[mask] for k, v in inputs.items()}
        #target_batch = [t for t, m in zip(target_batch, mask) if m]
        
        print("input_ids shape:", inputs["input_ids"].shape)
        print("max input_ids value:", inputs["input_ids"].max())
        print("vocabulary size:", tokenizer.vocab_size)

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device), 
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
       
        print(summaries.shape)
        print(inputs["attention_mask"].shape)
        # Finally, we decode the generated texts, 
        # replace the <n> token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=True) 
               for s in summaries]      
        
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
              
        metric.add_batch(predictions=decoded_summaries, references=target_batch)
        
    #  Finally compute and return the ROUGE scores.
    score = metric.compute(use_stemmer=True)
    return score

In [12]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

rouge_metric = load_metric('rouge')

<ipython-input-12-67199026c897>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')


In [13]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments


In [14]:
class PegasusDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])  # torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

In [15]:
def prepare_data(model_name, 
                 train_texts, train_labels, 
                 val_texts, val_labels, 
                 test_texts=None, test_labels=None):
  """
  Prepare input data for model fine-tuning
  """
  tokenizer = PegasusTokenizer.from_pretrained(model_name)

  prepare_val = False if val_texts is None or val_labels is None else True
  prepare_test = False if test_texts is None or test_labels is None else True

  def tokenize_data(texts, labels):
    encodings = tokenizer(texts, truncation=True, padding=True)
    decodings = tokenizer(labels, truncation=True, padding=True)
    dataset_tokenized = PegasusDataset(encodings, decodings)
    return dataset_tokenized

  train_dataset = tokenize_data(train_texts, train_labels)
  val_dataset = tokenize_data(val_texts, val_labels) if prepare_val else None
  test_dataset = tokenize_data(test_texts, test_labels) if prepare_test else None

  return train_dataset, val_dataset, test_dataset

In [44]:
def prepare_fine_tuning(model_name, train_dataset, val_dataset, freeze_encoder=False, output_dir='./results'):
  """
  Prepare configurations and base model for fine-tuning
  """
  torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

  if freeze_encoder:
    for param in model.model.encoder.parameters():
      param.requires_grad = False

  if val_dataset is not None:
    training_args = TrainingArguments(
      output_dir=output_dir,           
      adafactor=True,                  
      num_train_epochs=20,           
      per_device_train_batch_size=2,   
      per_device_eval_batch_size=2,   
      save_steps=5,                  
      save_total_limit=5,              
      evaluation_strategy='steps',     
      eval_steps=5,                  
      warmup_steps=500,  
      learning_rate=0.0002,              
      weight_decay=0.01,              
      logging_dir='./logs',            
      logging_steps=10,
      gradient_accumulation_steps=4
    )

    trainer = Trainer(
      model=model,                         
      args=training_args,                  
      train_dataset=train_dataset,         
      eval_dataset=val_dataset             
    )

  else:
    training_args = TrainingArguments(
      output_dir=output_dir,           # output directory
      adafactor=True,                  # use adafactor instead of AdamW
      num_train_epochs=1,           # total number of training epochs
      per_device_train_batch_size=1,   # batch size per device during training, can increase if memory allows
      save_steps=100,                  # number of updates steps before checkpoint saves
      save_total_limit=5,              # limit the total amount of checkpoints and deletes the older checkpoints
      warmup_steps=500,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               
      logging_dir='./logs',            
      logging_steps=10,
    )

    trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
    )

  return trainer

In [17]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/final/augmented_train3.csv')

In [18]:
df.shape

(480289, 2)

In [19]:
#importing the cleaned and preprocessed test csv file exported from jupyter notebook
test_df = pd.read_csv('/content/drive/MyDrive/final/cleaned_test.csv')

In [20]:
test_df.shape

(15441, 2)

In [21]:
#importing the cleaned and preprocessed test csv file exported from jupyter notebook
val_df = pd.read_csv('/content/drive/MyDrive/final/cleaned_val.csv')

In [22]:
val_df.shape

(15442, 2)

In [23]:
#shuffling all the rows so that original and augmented data rows gets mixed
train_df = df.sample(frac=1, random_state=42)

In [24]:
# reset index
train_df.reset_index(drop=True, inplace=True)

FOR ORIGINAL DATA WITHOUT AUGMENTATION

In [25]:
from datasets import load_dataset

In [26]:
dataset_cnn = load_dataset('cnn_dailymail','3.0.0')
split_lengths = [len(dataset_cnn[split])for split in dataset_cnn]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_cnn['train'].column_names}")

  0%|          | 0/3 [00:00<?, ?it/s]

Split lengths: [287113, 13368, 11490]
Features: ['article', 'highlights', 'id']


In [27]:
!pip install --upgrade accelerate
from functools import partial



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
# use cnn dataset as example, with first original as training data

train_texts, train_labels = dataset_cnn['train'][:30000]['article'], dataset_cnn['train']['highlights'][:30000]
#train_texts, train_labels = dataset['train']['document'], dataset['train']['summary']
val_texts, val_labels = dataset_cnn['validation']['article'][:2000], dataset_cnn['validation']['highlights'][:2000]

In [30]:
model_checkpoint = 'google/pegasus-multi_news'
# about 8 mins to finish for the whole dataset
train_dataset, val_dataset, _ = prepare_data(model_checkpoint, train_texts, train_labels, val_texts, val_labels)


In [45]:
%%time
# only about 1 minute
trainer = prepare_fine_tuning(model_checkpoint, train_dataset, val_dataset)

CPU times: user 15.1 s, sys: 4.2 s, total: 19.3 s
Wall time: 13.9 s


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1243: FutureWarning: `--adafactor` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--optim adafactor` instead
  warnings.warn(


In [46]:
trainer.train()

Step,Training Loss,Validation Loss
5,No log,8.673236
10,1.851900,8.652978
15,1.851900,8.621410
20,1.813200,8.564925


TrainOutput(global_step=20, training_loss=1.8325450897216797, metrics={'train_runtime': 195.9177, 'train_samples_per_second': 0.204, 'train_steps_per_second': 0.102, 'total_flos': 115578576568320.0, 'train_loss': 1.8325450897216797, 'epoch': 20.0})

In [35]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

rouge_metric = load_metric('rouge')

In [37]:
pegtokenizer = PegasusTokenizer.from_pretrained(model_checkpoint)

In [48]:
score = calculate_metric_on_test_ds(
    dataset_cnn['test'][:500], rouge_metric, trainer.model, pegtokenizer, batch_size = 3, column_text = 'article', column_summary= 'highlights'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

  0%|          | 0/167 [00:00<?, ?it/s]

input_ids shape: torch.Size([3, 1024])
max input_ids value: tensor(95399)
vocabulary size: 96103


  1%|          | 1/167 [00:24<1:07:06, 24.26s/it]

torch.Size([3, 128])
torch.Size([3, 1024])
input_ids shape: torch.Size([3, 1024])
max input_ids value: tensor(88480)
vocabulary size: 96103


  1%|          | 2/167 [00:47<1:05:07, 23.68s/it]

...

 99%|█████████▉| 166/167 [1:03:55<00:23, 23.15s/it]

torch.Size([3, 128])
torch.Size([3, 1024])
input_ids shape: torch.Size([2, 1024])
max input_ids value: tensor(94693)
vocabulary size: 96103


100%|██████████| 167/167 [1:04:12<00:00, 23.07s/it]

torch.Size([2, 128])
torch.Size([2, 1024])


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.27176,0.096749,0.180235,0.213639
